# Análise dos dados com status de sucesso

In [1]:
import pandas as pd

df = pd.read_csv('traefik.csv',sep=',')
df = df[df['status'] == '200']

### Existe algum padrão de horário?

In [3]:
df['data1'] = pd.to_datetime(df.data1, format='%Y-%m-%d %H:%M:%S')

In [4]:
hours = pd.DataFrame()
hours = pd.DataFrame(columns=['Horário', 'Qnt. de Acessos'])

In [5]:
for i in range(24):
    hours.loc[i]=[ i, df.data1[df['data1'].dt.hour == i].count()]

In [6]:
hours

,Horário,Qnt. de Acessos
0,0,2294
1,1,1627
2,2,1559
3,3,1320
4,4,1622
5,5,1804
6,6,2091
7,7,11739
8,8,41801
9,9,39686


In [7]:
dateAcess = df.groupby([df['data1'].dt.month, df['data1'].dt.year, ]).count().sort_index()
dateAcess.rename(columns={'status': 'Acessos'}, inplace=True)
dateAcess = dateAcess.Acessos
dateAcess = dateAcess.rename_axis(['Mês', 'Ano']).reset_index()
dateAcess

,Mês,Ano,Acessos
0,1,2022,35663
1,2,2022,23834
2,3,2022,57984
3,4,2022,47523
4,5,2022,74346
5,6,2022,105878
6,7,2022,40353
7,8,2022,11765
8,9,2021,975
9,10,2021,654


### Algum IP específico?

In [8]:
ips = df.groupby(['ip']).count()
ips.rename(columns={'status': 'acessos'}, inplace=True)
ips = ips.acessos

In [9]:
ips.sort_values(ascending=False)

ip
200.129.47.254     39717
216.245.221.89     25251
191.5.210.64       12273
10.0.84.198         8224
172.18.0.1          8118
                   ...  
114.119.135.196        1
114.119.135.195        1
114.119.135.193        1
114.119.135.182        1
95.217.8.54            1
Name: acessos, Length: 4929, dtype: int64

### Padrões de recursos?

In [10]:
recurso = df.groupby(["recurso"]).count()
recurso.rename(columns={'status': 'acessos'}, inplace=True)
recurso = recurso.acessos.reset_index()

In [11]:
recurso.sort_values(by='acessos', ascending=False)

,recurso,acessos
33859,/main/,25292
33856,/favicon.ico,4597
35109,/pesquisa/meusProjetos,4196
82,/avaliacao/index.php/432657,4157
4618,/computacao/course/view.php?id=27,3956
...,...,...
43467,/pesquisa/static/aprovado.png?rnd=9039,1
43468,/pesquisa/static/aprovado.png?rnd=9040,1
9950,/computacao/lib/ajax/service.php?sesskey=7ZNhb...,1
43470,/pesquisa/static/aprovado.png?rnd=9042,1


In [24]:
dbrecurso = pd.read_csv('recurso.csv', sep=',')
dbrecurso = dbrecurso[dbrecurso['status'] == '200']

In [25]:
qtdrecursos = dbrecurso.groupby(['status', 'recurso']).count()
qtdrecursos.rename(columns={'data1': 'acessos'}, inplace=True)
qtdrecursos = qtdrecursos.acessos

In [26]:
qtdrecursos.sort_values(ascending=False)

status  recurso     
200     computacao      222833
        pesquisa         76438
        avaliacao        55178
        main             25427
        phpmyadmin       20995
        docs             19980
        yoko             13751
        eleicoes          2059
        cobras            1743
        cppgi             1385
        cgs               1315
        cieco              100
        facd                94
        webapi              41
        .well-known         16
        public              13
        db                   5
        chaves               3
        mongo                2
        Autodiscover         1
        evox                 1
        hestia               1
        index.php?s=         1
Name: acessos, dtype: int64

### Quantos métodos diferentes?

In [12]:
metodos = df.groupby(['status', 'metodo']).count()
metodos.rename(columns={'data1': 'acessos'}, inplace=True)
metodos = metodos.acessos
metodos = metodos.reset_index()
metodos.sort_values(by='acessos', ascending=False)

,status,metodo,acessos
0,200,GET,321667
2,200,POST,100149
1,200,HEAD,25371


## Gráficos

In [13]:
import plotly.offline as py
import plotly.graph_objs as go

py.init_notebook_mode(connected=True)

### Acessos por data:

In [14]:
graf2021 = go.Scatter(x=['Setembro', 'Outubro', 'Dezembro'],
                    y=dateAcess['Acessos'][8:],
                    name='2021')

graf2022 = go.Scatter(x=['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto'],
                    y=dateAcess['Acessos'][0:8],
                    name='2022')

layout = go.Layout(title='Acessos por Data',
                   yaxis={'title':'Quantidade de Acessos'},
                   xaxis={'title': 'Mês de Acesso'})

fig = go.Figure(data=[graf2021, graf2022], layout=layout)

py.iplot(fig)

### Acessos por método:

In [15]:
grafMetodo = go.Bar(x = metodos['metodo'],
                  y = metodos['acessos'])

layout = go.Layout(title='Acessos por Métodos',
                   yaxis={'title':'Quantidade de Acessos'},
                   xaxis={'title': 'Métodos'})

fig = go.Figure(data=grafMetodo, layout=layout)

py.iplot(fig)